## 1.환경준비

### (1) 라이브러리 불러오기

In [38]:
import joblib
import pandas as pd
import numpy as np
import gensim
import tensorflow as tf
import warnings 
warnings.filterwarnings('ignore')

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from keras.backend import clear_session
from keras.layers import Input, Dense, Embedding, LSTM
from keras.models import Model
from keras.optimizers import Adam

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics.pairwise import cosine_similarity


### (2) 데이터 로딩

In [39]:
dataset = joblib.load('/Users/hyeontae/Desktop/AIVLE-School-FAQ-ChatBot/dataset.pkl')

## 2.Modeling

### (1) train/test dataset 분리
   * test dataset : intent마다 무작위로 질문 2개를 뽑아 test 데이터로 분리
   * train dataset : 나머지 데이터


In [40]:
# test dataset
test_dataset = dataset.loc[dataset['intent'] == 1].sample(n=2, random_state = 42)

for i in range(2, dataset['intent'].max()+1) :
    test_dataset = pd.concat([test_dataset, dataset.loc[dataset['intent'] == i].sample(n=2, random_state = 42)])

# train dataset
train_dataset = dataset.drop(test_dataset.index)

# 인덱스 초기화
train_dataset = train_dataset.reset_index(drop = True)
test_dataset = test_dataset.reset_index(drop = True)

### (2) STEP 1 : Type 분류 모델링(LSTM)


#### 1) 데이터 준비

- label

In [41]:
y_train = train_dataset['type']
y_test = test_dataset['type']

- 토큰화

In [42]:
# 각각의 토큰에 인덱스를 부여하는 토크나이저 생성
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_dataset['Q'])

# 전체 토큰의 수로 vocab_size 지정
vocab_size = len(tokenizer.index_word) + 1

# 토크나이즈 된 데이터를 가지고 모두 시퀀스로 변환
x_train = tokenizer.texts_to_sequences(train_dataset['Q'])
x_test = tokenizer.texts_to_sequences(test_dataset['Q'])

- Padding

In [43]:
max_sequence_length = max(train_dataset['Q'].apply(lambda x : len(x)))

x_train= pad_sequences(x_train, maxlen = max_sequence_length)
x_test = pad_sequences(x_test, maxlen= max_sequence_length)

#### 2) Modeling

In [44]:
# 1. 이전 세션 클리어
clear_session
# 2. 모델 엮기
il = Input(shape = (x_train.shape[1],))

el = Embedding(input_dim = vocab_size,
               output_dim = 256,
               input_length = max_sequence_length)(il)
lstm_l = LSTM(128, return_sequences= False)(el)
ol = Dense(1, activation = 'sigmoid')(lstm_l)

# 3. 모델 선언
model = Model(il, ol)
# 4. 모델 컴파일
model.compile(loss = 'binary_crossentropy', metrics = ['accuracy'], optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.01))
# 5. 모델 요약
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 144)]             0         
                                                                 
 embedding_3 (Embedding)     (None, 144, 256)          343040    
                                                                 
 lstm_3 (LSTM)               (None, 128)               197120    
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 540289 (2.06 MB)
Trainable params: 540289 (2.06 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [45]:
from keras.callbacks import EarlyStopping

es = EarlyStopping(monitor = 'val_loss',
                   min_delta = 0,
                   patience = 5,
                   verbose = 1,
                   restore_best_weights = True)

In [46]:
model.fit(x_train, y_train, epochs = 1000, callbacks = [es], validation_split = 0.2, verbose = 1)

Epoch 1/1000
30/30 [==============================] - 4s 103ms/step - loss: 0.1952 - accuracy: 0.9024 - val_loss: 0.1482 - val_accuracy: 0.9623
Epoch 2/1000
30/30 [==============================] - 3s 103ms/step - loss: 7.2214e-05 - accuracy: 1.0000 - val_loss: 0.1846 - val_accuracy: 0.9623
Epoch 3/1000
30/30 [==============================] - 3s 99ms/step - loss: 2.9502e-05 - accuracy: 1.0000 - val_loss: 0.2054 - val_accuracy: 0.9623
Epoch 4/1000
30/30 [==============================] - 3s 114ms/step - loss: 2.1399e-05 - accuracy: 1.0000 - val_loss: 0.2210 - val_accuracy: 0.9623
Epoch 5/1000
30/30 [==============================] - 3s 107ms/step - loss: 1.7408e-05 - accuracy: 1.0000 - val_loss: 0.2339 - val_accuracy: 0.9623
Epoch 6/1000
30/30 [==============================] - 3s 101ms/step - loss: 1.4580e-05 - accuracy: 1.0000 - val_loss: 0.2426 - val_accuracy: 0.9582
Epoch 6: early stopping


#### 3) Evaluation

In [47]:
y_pred = model.predict(x_test)
y_pred_binary = np.where(y_pred > 0.5, 1, 0)

print(classification_report(y_test, y_pred_binary))

4/4 [==============================] - 0s 22ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        60
           1       1.00      1.00      1.00        46

    accuracy                           1.00       106
   macro avg       1.00      1.00      1.00       106
weighted avg       1.00      1.00      1.00       106



### (3) STEP 2 : Word Embedding(Word2Vec)

* Pre-trained Word2Vec model
    * ko.bin
* 사전학습 모델을 로딩하고, train 데이터셋의 질문(Q)을 임베딩벡터로 만들어, 열(Column)로 추가

#### 1) Model Loading


In [48]:
pre_wv_model = gensim.models.Word2Vec.load('/Users/hyeontae/Desktop/AIVLE-School-FAQ-ChatBot/ko/ko.bin')

In [49]:
# 모델의 벡터크기 조회
size = pre_wv_model.vector_size

#### 2) train_data에 임베딩벡터 결과 저장

- 함수 정의

In [50]:
# Word2Vec 모델로부터 하나의 문장을 벡터화 시키는 함수
def get_sent_embedding(model, embedding_size, tokenized_words):
    # 임베딩 벡터를 0으로 초기화
    feature_vec = np.zeros((embedding_size,), dtype='float32')
    # 단어 개수 초기화
    n_words = 0
    # 모델 단어 집합 생성
    index2word_set = set(model.wv.index2word)
    # 문장의 단어들을 하나씩 반복
    for word in tokenized_words:
        # 모델 단어 집합에 해당하는 단어일 경우에만
        if word in index2word_set:
            # 단어 개수 1 증가
            n_words += 1
            # 임베딩 벡터에 해당 단어의 벡터를 더함
            feature_vec = np.add(feature_vec, model[word])
    # 단어 개수가 0보다 큰 경우 벡터를 단어 개수로 나눠줌 (평균 임베딩 벡터 계산)
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

- train_data의 질문별 임베딩 결과 저장

In [51]:
train_dataset['Q_embedding'] = train_dataset.Q.apply(lambda x : get_sent_embedding(pre_wv_model, size, x))

### (3) ChatBot 구축

#### 1) 함수 생성

In [52]:
def ChatBot(question):
    input = [list(question.split(' '))]
    input = tokenizer.texts_to_sequences(input)
    input = pad_sequences(input, maxlen= max_sequence_length)

    y_pred = model.predict(input)
    y_pred = np.where(y_pred > 0.5, 1, 0)
    data = train_dataset.loc[train_dataset['type'] == y_pred[0][0]]
    data = data.reset_index(drop= True)

    vector = get_sent_embedding(pre_wv_model, 200, question)
    data_vector = list(data['Q_embedding'])
    pred_classes = np.argmax(cosine_similarity([vector], data_vector))

    answers = data['A'].loc[data['intent'] == data['intent'][pred_classes]]
    print(f'질문 : {question}')
    print(f'대답 : {np.random.choice(answers)}')
    return

#### 2) Test

In [53]:
for text in test_dataset['Q'] :
    ChatBot(text)
    print('----------------------------------------------------------------------------------------------------------------------------------------')

1/1 [==============================] - 0s 12ms/step
질문 : 떨어뜨려서 핸드폰 액정 나갔어
대답 : as 맡기세요.
----------------------------------------------------------------------------------------------------------------------------------------
1/1 [==============================] - 0s 34ms/step
질문 : 액정 나갔어
대답 : 술 안 마셔도 놀 수 있어요.
----------------------------------------------------------------------------------------------------------------------------------------
1/1 [==============================] - 0s 18ms/step
질문 : 내 의지 는 상관없나 봐
대답 : 제가 있잖아요.
----------------------------------------------------------------------------------------------------------------------------------------
1/1 [==============================] - 0s 21ms/step
질문 : 내 의지 로 안되는 일인 가 봐
대답 : 맛있게 드세요.
----------------------------------------------------------------------------------------------------------------------------------------
1/1 [==============================] - 0s 16ms/step
질문 : 코 가 막혀서 싫어
대답 : 감기 조심하세요.
---------------------